In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

# jax.config.update("jax_debug_nans", True)

2025-03-01 21:30:17.080891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740861017.094625  432763 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740861017.098653  432763 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [4]:
def my_env_creator(params, battery_type):
    env = RECEnv(params, battery_type)
    return env

In [5]:
from ernestogym.envs_jax.multi_agent.utils import parameter_generator

In [6]:
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_train.yaml"
# world = "ernestogym/envs_jax/single_agent/world_fading.yaml"

# battery_type = 'fading'
# battery_type = 'degrading'
battery_type = 'degrading_dropflow'

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params

{'num_battery_agents': 3,
 'num_passive_houses': 0,
 'batteries': [{'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 288.0,
    'temp_ambient': 298.15,
    'nominal_cost': 3000.0},
   'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
    'current': {'low': -180.0, 'high': 60.0},
    'power': {'low': -71712.0, 'high': 23904.0},
    'temperature': {'low': 273.15, 'high': 323.15},
    'temp_ambient': {'low': 273.15, 'high': 323.15},
    'soc': {'low': 0.2, 'high': 1.0},
    'soh': {'low': 0.6, 'high': 1.0}},
   'init': {'voltage': 310.8,
    'current': 0.0,
    'power': 0.0,
    'temperature': 293.15,
    'temp_ambient': 293.15,
    'soc': 0.2,
    'soh': 1.0}},
  {'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 2

In [7]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# # ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
# r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# # bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
# world = "ernestogym/envs_jax/single_agent/ijcnn_deg_test.yaml"
# # world = "ernestogym/envs_jax/single_agent/world_fading.yaml"
#
# params_testing = parameter_generator(
#     input_var='current',
#     battery_options=pack_options,
#     electrical_model=ecm,
#     thermal_model=r2c,
#     aging_model=bolun,
#     world_options=world,
#     use_reward_normalization=True
#
# )
#
# params_testing['step']

In [8]:
from algorithms.multi_agent_ppo_vectorized import make_train, train        #, train_for, train_for_flax
from flax.core.frozen_dict import freeze, unfreeze

In [9]:
num_envs = 4

config = {
    "LR_BATTERIES": 1e-3,
    "LR_REC": 1e-3,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": 876000 * num_envs,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 32,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_EPS": 0.2,
    "ENT_COEF": 0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "ANNEAL_LR": True,
    'DEBUG': False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": 'normal',
    'NETWORK': 'actor_critic'
}

env_type = config['ENV_TYPE']

rng = jax.random.PRNGKey(42)

env = my_env_creator(params, battery_type)
env, train_state = make_train(config, env)

# env_testing, env_testing_params = my_env_creator(params_testing, battery_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*8

# out = train_jit(rng)

config = freeze(config)

In [10]:
env.battery_agents

['battery_agent_0', 'battery_agent_1', 'battery_agent_2']

In [11]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [12]:
networks_batteries

WrappedStackedActorCritic(
  vmappd_module=StackedActorCritic(
    act_layers=[Linear(
      bias=Param(
        value=Array(shape=(3, 64), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x7b41dc546840>,
      dot_general=<function dot_general at 0x7b4298a79e40>,
      dtype=None,
      in_features=10,
      kernel=Param(
        value=Array(shape=(3, 10, 64), dtype=float32)
      ),
      kernel_init=<function orthogonal.<locals>.init at 0x7b41dc546340>,
      out_features=64,
      param_dtype=<class 'jax.numpy.float32'>,
      precision=None,
      use_bias=True
    ), <PjitFunction of <function tanh at 0x7b42987547c0>>, Linear(
      bias=Param(
        value=Array(shape=(3, 32), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x7b41dc37ca40>,
      dot_general=<function dot_general at 0x7b4298a79e40>,
      dtype=None,
      in_features=64,
      kernel=Param(
        value=Array(shape=(3, 64, 32), dtype=float32)
      ),
  

In [13]:
network_rec

RECActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(64,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7b41dc5a3b00>,
    dot_general=<function dot_general at 0x7b4298a79e40>,
    dtype=None,
    in_features=12,
    kernel=Param(
      value=Array(shape=(12, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7b41dc41b6a0>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x7b42987547c0>>, Linear(
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7b41dc27d260>,
    dot_general=<function dot_general at 0x7b4298a79e40>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7b41dc27d440>,
    out_features=32,
    param_dtype=<clas

In [ ]:
import time
import matplotlib.pyplot as plt
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
out = jax.block_until_ready(train(env, config, train_state, rng, validate=False)) #, freq_val=10, val_env=env_testing, val_params=env_testing_params, val_rng=val_rng, val_num_iters=val_num_iters))

train_state = out['runner_state'][0]

print(f"time: {time.time() - t0:.2f} s")
# plt.plot(out["metrics"]["returned_episode_returns"].mean(-1).reshape(-1))
# plt.xlabel("Update Step")
# plt.ylabel("Return")
# plt.show()

In [15]:
jax.tree.map(lambda x: x.shape, out['metrics'])

{'actions': {'REC_agent': (106, 8192, 4, 3),
  'battery_agent_0': (106, 8192, 4, 1),
  'battery_agent_1': (106, 8192, 4, 1),
  'battery_agent_2': (106, 8192, 4, 1)},
 'buy_prices': (106, 8192, 4, 3),
 'demands': (106, 8192, 4, 3),
 'generations': (106, 8192, 4, 3),
 'norm_reward': {'r_clipping': (106, 8192, 4, 3),
  'r_deg': (106, 8192, 4, 3),
  'r_op': (106, 8192, 4, 3),
  'r_trad': (106, 8192, 4, 3)},
 'pure_reward': {'r_clipping': (106, 8192, 4, 3),
  'r_deg': (106, 8192, 4, 3),
  'r_op': (106, 8192, 4, 3),
  'r_trad': (106, 8192, 4, 3)},
 'r_glob': (106, 8192, 4, 3),
 'r_tot': (106, 8192, 4, 3),
 'rec_reward': (106, 8192, 4),
 'self_consumption': (106, 8192, 4),
 'sell_prices': (106, 8192, 4, 3),
 'soc': (106, 8192, 4, 3),
 'soh': (106, 8192, 4, 3),
 'tot_incentives': (106, 8192, 4),
 'weig_reward': {'r_clipping': (106, 8192, 4, 3),
  'r_deg': (106, 8192, 4, 3),
  'r_op': (106, 8192, 4, 3),
  'r_trad': (106, 8192, 4, 3)}}

In [16]:
jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

{'actions': {'REC_agent': Array(True, dtype=bool),
  'battery_agent_0': Array(True, dtype=bool),
  'battery_agent_1': Array(True, dtype=bool),
  'battery_agent_2': Array(True, dtype=bool)},
 'buy_prices': Array(False, dtype=bool),
 'demands': Array(False, dtype=bool),
 'generations': Array(False, dtype=bool),
 'norm_reward': {'r_clipping': Array(True, dtype=bool),
  'r_deg': Array(True, dtype=bool),
  'r_op': Array(False, dtype=bool),
  'r_trad': Array(True, dtype=bool)},
 'pure_reward': {'r_clipping': Array(True, dtype=bool),
  'r_deg': Array(True, dtype=bool),
  'r_op': Array(False, dtype=bool),
  'r_trad': Array(True, dtype=bool)},
 'r_glob': Array(True, dtype=bool),
 'r_tot': Array(True, dtype=bool),
 'rec_reward': Array(True, dtype=bool),
 'self_consumption': Array(False, dtype=bool),
 'sell_prices': Array(False, dtype=bool),
 'soc': Array(True, dtype=bool),
 'soh': Array(True, dtype=bool),
 'tot_incentives': Array(False, dtype=bool),
 'weig_reward': {'r_clipping': Array(True, dty

In [17]:
jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

{'actions': {'REC_agent': Array([False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True], dtype=bool),
  'battery_agent_0': Array([False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True

In [18]:
out['metrics']['actions']['REC_agent'][0]

Array([[[6.18245184e-01, 1.15211066e-02, 3.70233715e-01],
        [4.76111680e-01, 1.07430793e-01, 4.16457564e-01],
        [6.46410557e-03, 1.63270324e-03, 9.91903245e-01],
        [2.42251217e-01, 3.14746439e-01, 4.43002373e-01]],

       [[8.96850526e-01, 8.94142613e-02, 1.37352385e-02],
        [1.17478166e-02, 8.62250011e-03, 9.79629636e-01],
        [5.90783834e-01, 3.90382439e-01, 1.88337564e-02],
        [1.64664492e-01, 3.44788134e-01, 4.90547359e-01]],

       [[2.54506017e-06, 1.45207599e-01, 8.54789913e-01],
        [6.51003793e-02, 7.17693985e-01, 2.17205614e-01],
        [3.98322307e-02, 9.60167825e-01, 4.01982143e-15],
        [6.44524172e-02, 8.75886798e-01, 5.96608035e-02]],

       ...,

       [[1.25405435e-02, 5.48078381e-02, 9.32651579e-01],
        [1.70528516e-02, 8.48690152e-01, 1.34257033e-01],
        [4.69850957e-01, 4.54230815e-01, 7.59181976e-02],
        [6.41964495e-01, 1.65383399e-01, 1.92652106e-01]],

       [[1.22900240e-01, 6.68265462e-01, 2.08834291

In [19]:
out['metrics']['actions']['REC_agent'][1]

Array([[[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       ...,

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]]], dtype=float32)

In [20]:
np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

True

In [21]:
jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

{'actions': {'REC_agent': (array([  1,   1,   1, ..., 105, 105, 105]),
   array([   0,    0,    0, ..., 8191, 8191, 8191]),
   array([0, 0, 0, ..., 3, 3, 3]),
   array([0, 1, 2, ..., 0, 1, 2])),
  'battery_agent_0': (array([  2,   2,   2, ..., 105, 105, 105]),
   array([   0,    0,    0, ..., 8191, 8191, 8191]),
   array([0, 1, 2, ..., 1, 2, 3]),
   array([0, 0, 0, ..., 0, 0, 0])),
  'battery_agent_1': (array([  2,   2,   2, ..., 105, 105, 105]),
   array([   0,    0,    0, ..., 8191, 8191, 8191]),
   array([0, 1, 2, ..., 1, 2, 3]),
   array([0, 0, 0, ..., 0, 0, 0])),
  'battery_agent_2': (array([  2,   2,   2, ..., 105, 105, 105]),
   array([   0,    0,    0, ..., 8191, 8191, 8191]),
   array([0, 1, 2, ..., 1, 2, 3]),
   array([0, 0, 0, ..., 0, 0, 0]))},
 'buy_prices': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),
 'demands': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),

In [22]:
out['metrics']['actions']['REC_agent']

Array([[[[6.18245184e-01, 1.15211066e-02, 3.70233715e-01],
         [4.76111680e-01, 1.07430793e-01, 4.16457564e-01],
         [6.46410557e-03, 1.63270324e-03, 9.91903245e-01],
         [2.42251217e-01, 3.14746439e-01, 4.43002373e-01]],

        [[8.96850526e-01, 8.94142613e-02, 1.37352385e-02],
         [1.17478166e-02, 8.62250011e-03, 9.79629636e-01],
         [5.90783834e-01, 3.90382439e-01, 1.88337564e-02],
         [1.64664492e-01, 3.44788134e-01, 4.90547359e-01]],

        [[2.54506017e-06, 1.45207599e-01, 8.54789913e-01],
         [6.51003793e-02, 7.17693985e-01, 2.17205614e-01],
         [3.98322307e-02, 9.60167825e-01, 4.01982143e-15],
         [6.44524172e-02, 8.75886798e-01, 5.96608035e-02]],

        ...,

        [[1.25405435e-02, 5.48078381e-02, 9.32651579e-01],
         [1.70528516e-02, 8.48690152e-01, 1.34257033e-01],
         [4.69850957e-01, 4.54230815e-01, 7.59181976e-02],
         [6.41964495e-01, 1.65383399e-01, 1.92652106e-01]],

        [[1.22900240e-01, 6.682654

In [23]:
out['metrics']['pure_reward']['r_clipping'][3]

Array([[[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       ...,

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]]], dtype=float32)

In [24]:
network_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)
network_batteries

WrappedStackedActorCritic(
  vmappd_module=StackedActorCritic(
    act_layers=[Linear(
      bias=Param(
        value=Array(shape=(3, 64), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x71662c24e8e0>,
      dot_general=<function dot_general at 0x7166ded65e40>,
      dtype=None,
      in_features=10,
      kernel=Param(
        value=Array(shape=(3, 10, 64), dtype=float32)
      ),
      kernel_init=<function orthogonal.<locals>.init at 0x71662c24e3e0>,
      out_features=64,
      param_dtype=<class 'jax.numpy.float32'>,
      precision=None,
      use_bias=True
    ), <PjitFunction of <function silu at 0x7166de4b80e0>>, Linear(
      bias=Param(
        value=Array(shape=(3, 32), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x7166205b4ae0>,
      dot_general=<function dot_general at 0x7166ded65e40>,
      dtype=None,
      in_features=64,
      kernel=Param(
        value=Array(shape=(3, 64, 32), dtype=float32)
      ),
  

## State saving

In [25]:
save_state_multiagent(network_batteries, network_rec, config, params, env_type=env_type, additional_info='with sigmoid in rec network')

In [26]:
out

{'metrics': {'actions': {'REC_agent': Array([[[[6.18245184e-01, 1.15211066e-02, 3.70233715e-01],
            [4.76111680e-01, 1.07430793e-01, 4.16457564e-01],
            [6.46410557e-03, 1.63270324e-03, 9.91903245e-01],
            [2.42251217e-01, 3.14746439e-01, 4.43002373e-01]],
   
           [[8.96850526e-01, 8.94142613e-02, 1.37352385e-02],
            [1.17478166e-02, 8.62250011e-03, 9.79629636e-01],
            [5.90783834e-01, 3.90382439e-01, 1.88337564e-02],
            [1.64664492e-01, 3.44788134e-01, 4.90547359e-01]],
   
           [[2.54506017e-06, 1.45207599e-01, 8.54789913e-01],
            [6.51003793e-02, 7.17693985e-01, 2.17205614e-01],
            [3.98322307e-02, 9.60167825e-01, 4.01982143e-15],
            [6.44524172e-02, 8.75886798e-01, 5.96608035e-02]],
   
           ...,
   
           [[1.25405435e-02, 5.48078381e-02, 9.32651579e-01],
            [1.70528516e-02, 8.48690152e-01, 1.34257033e-01],
            [4.69850957e-01, 4.54230815e-01, 7.59181976e-02],
